In [1]:
from bs4 import BeautifulSoup, Tag
import numpy as np
import pandas as pd
import torch
import constants
import secrets
import re
import requests
try:
    # For Python 3.0 and later
    from urllib.request import urlopen
except ImportError:
    # Fall back to Python 2's urllib2
    from urllib2 import urlopen

import certifi
import json

c:\Users\trexx\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_jsonparsed_data(url):

    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)

In [3]:
url = (f"{constants.FMP_URL}{constants.STOCK_NEWS}?tickers=AAPL&apikey={secrets.FMP_API_KEY}")
print(get_jsonparsed_data(url))

C:\Users\trexx\AppData\Local\Temp/ipykernel_17196/1748608114.py:3: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


[{'symbol': 'AAPL', 'publishedDate': '2023-06-14 17:10:41', 'title': "Dow Jones Today: Index Falls on Fed's ‘Pause'", 'image': 'https://cdn.snapi.dev/images/v1/g/t/gettyimages-1258692541-321d445ea8a84f93a54340ae919ed7a2-1934694.jpg', 'site': 'Investopedia', 'text': 'The Dow fell 230 points after the Fed holds rates steady, but hints more rate hikes to come', 'url': 'https://www.investopedia.com/dow-jones-today-june-14-2023-7546822'}, {'symbol': 'AAPL', 'publishedDate': '2023-06-14 15:18:48', 'title': 'Apple: Major Market Correction Likely Ahead', 'image': 'https://cdn.snapi.dev/images/v1/1/l/image-1399956287-1934455.jpg', 'site': 'Seeking Alpha', 'text': 'Apple Inc. is a high-quality company capable of huge profitability and sustained growth. A poor start to 2023 seems to have gone ignored by investors, with dropping net incomes and margins being drowned out by hype and momentum. Current valuations leave little to be desired for value-oriented investors. I believe the 20% premium curre

In [4]:
df = pd.read_csv(constants.DATA_PATH)
print(df.head())

Empty DataFrame
Columns: [date,  aapl]
Index: []
